In [1]:
# ok first some imports
import pyvis as pv 
import networkx as nx 
import pandas as pd
import requests
import scipy
from bs4 import BeautifulSoup
from pyvis.network import Network

import requests
import os
import re
import numpy as np
import gensim
import gensim.corpora as corpora

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from pprint import pprint
from bs4 import BeautifulSoup
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sathishkumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Functions for graphing

In [14]:
def add_edges_rey(graph, edges):
    """
    This method serves to add multiple edges between existing nodes
    in the network instance. Adding of the edges is done based off
    of the IDs of the nodes. Order does not matter unless dealing with a
    directed graph.

    :param edges: A list of tuples, each tuple consists of source of edge,
                  edge destination and and optional width.

    :type arrowStrikethrough: list of tuples
    """
    for edge in edges:
        # if incoming tuple contains a weight
        if len(edge) == 3:
            graph.add_edge(edge[0], edge[1], color=edge[2], arrows = "hi")
        else:
            graph.add_edge(edge[0], edge[1])


def connect_edges(edges_list, graph, color):
    connect_list = [(story1, story2, color) for story1 in edges_list for story2 in edges_list if story1!=story2 and story1>story2]
    print(connect_list)
    add_edges_rey(graph, connect_list)

Functions for text analysis

In [15]:
def clean_story(URL):

    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find(id="container")
    main_text = results.find_all("p")

    full_story = ""
    for element in main_text:
        clean_element = element.text.strip()
        clean_element = re.sub(r'[0-9]+', '', clean_element)
        clean_element = re.sub(r'\[|\]|\t|\n', '', clean_element)
        full_story += clean_element
    return full_story

def two_digit(digit):
    if (digit < 10):
        digit = f"0{digit}"
    return str(digit)

def return_URL(day, story):
    story = two_digit(story)
    day = two_digit(day)
    URL = f"https://www.brown.edu/Departments/Italian_Studies/dweb/texts/DecShowText.php?myID=nov{day}{story}&lang=eng"
    return URL

def analyze_story():
    #TODO
    return

## Graphing

### Colors and Dictionaries

In [16]:
# creating storytellers colors
storytellers = {
    "Panfilo" : "#9e0142",
    "Neifile" : "#d53e4f",
    "Filomena" : "#f46d43",
    "Dioneo" : "#fdae61",
    "Fiametta" : "#fee08b",
    "Emilia" : "#e6f598",
    "Filostrato" : "#abdda4",
    "Lauretta" : "#66c2a5",
    "Elissa" : "#3288bd",
    "Pampinea" : "#5e4fa2"
}

# themes

THEMES = ["Storytelling", "Cultural_Exchange", "Sexuality", "Gender_Bending"]
theme_dict = {}
color_coding = {
    "Storytelling" : "blue",
    "Cultural_Exchange" : "green",
    "Sexuality" : "red",
    "Gender_Bending": "purple"
}

In [20]:
## ACTUAL STUFF
# read in data and create graph
df = pd.read_csv("decameron_themes.csv")
main_graph = Network(directed = True)
main_graph.set_edge_smooth("dynamic")

for index, row in df.iterrows():

    # this adds nodes to the graphs of each of the storytellers
    main_graph.add_node(row["Story_Hash"], label = f"Day {row["Day"]}, Story {row["Story"]}", color = storytellers[row["Storyteller"]], title='<a href=\'http://www.google.com\'>google</a>')

    # TODO main_graph[row[""]= 
    #


# creating theme lists
for theme in THEMES:
    theme_dict[theme] = df.loc[df[theme] == True]["Story_Hash"].tolist()
    connect_edges(theme_dict[theme], main_graph, color_coding[theme])


## OK LETS SEE THIS
main_graph.barnes_hut(spring_length = 150)
main_graph.show_buttons()
main_graph.show("hello.html")

[(30, 1, 'blue'), (30, 17, 'blue'), (30, 19, 'blue'), (30, 26, 'blue'), (17, 1, 'blue'), (19, 1, 'blue'), (19, 17, 'blue'), (26, 1, 'blue'), (26, 17, 'blue'), (26, 19, 'blue')]
[(30, 3, 'green'), (30, 17, 'green'), (30, 19, 'green'), (17, 3, 'green'), (19, 3, 'green'), (19, 17, 'green')]
[(30, 17, 'red'), (30, 26, 'red'), (26, 17, 'red')]
[]


## Text Analysis

In [3]:
# this creates a 2d array of all the stories
stories_all = []
for day in np.arange(1, 11, 1):
    day_stories = []
    for story in np.arange(1, 11, 1):
        print(f"Currently Scraping: Day {day}, Story {story}")
        day_stories.append(clean_story(return_URL(day, story)))
    stories_all.append(day_stories)

Currently Scraping: Day 1, Story 1
Currently Scraping: Day 1, Story 2
Currently Scraping: Day 1, Story 3
Currently Scraping: Day 1, Story 4
Currently Scraping: Day 1, Story 5
Currently Scraping: Day 1, Story 6
Currently Scraping: Day 1, Story 7
Currently Scraping: Day 1, Story 8
Currently Scraping: Day 1, Story 9
Currently Scraping: Day 1, Story 10
Currently Scraping: Day 2, Story 1
Currently Scraping: Day 2, Story 2
Currently Scraping: Day 2, Story 3
Currently Scraping: Day 2, Story 4
Currently Scraping: Day 2, Story 5
Currently Scraping: Day 2, Story 6
Currently Scraping: Day 2, Story 7
Currently Scraping: Day 2, Story 8
Currently Scraping: Day 2, Story 9
Currently Scraping: Day 2, Story 10
Currently Scraping: Day 3, Story 1
Currently Scraping: Day 3, Story 2
Currently Scraping: Day 3, Story 3
Currently Scraping: Day 3, Story 4
Currently Scraping: Day 3, Story 5
Currently Scraping: Day 3, Story 6
Currently Scraping: Day 3, Story 7
Currently Scraping: Day 3, Story 8
Currently Scraping

In [12]:
stop_words = stopwords.words("english")
annoying_words = ["thou", "one", "said", "thy", "tis", "thee"]
stop_words += annoying_words
print(stop_words)
texts = [[word for word in simple_preprocess(str(story)) if word not in stop_words] for story in stories_all]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
#num_topics = 10
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=42, passes=50, alpha="auto", per_word_topics=True)

In [13]:
pprint(lda_model.print_topics())

[(0,
  '0.008*"lady" + 0.006*"would" + 0.005*"might" + 0.004*"love" + 0.004*"time" '
  '+ 0.004*"made" + 0.004*"great" + 0.004*"man" + 0.004*"well" + 0.004*"come"'),
 (1,
  '0.000*"lady" + 0.000*"would" + 0.000*"well" + 0.000*"might" + 0.000*"love" '
  '+ 0.000*"man" + 0.000*"wherefore" + 0.000*"good" + 0.000*"made" + '
  '0.000*"upon"'),
 (2,
  '0.000*"would" + 0.000*"might" + 0.000*"man" + 0.000*"lady" + 0.000*"messer" '
  '+ 0.000*"made" + 0.000*"well" + 0.000*"time" + 0.000*"upon" + 0.000*"love"'),
 (3,
  '0.000*"lady" + 0.000*"would" + 0.000*"made" + 0.000*"love" + 0.000*"great" '
  '+ 0.000*"might" + 0.000*"wherefore" + 0.000*"never" + 0.000*"man" + '
  '0.000*"may"'),
 (4,
  '0.010*"messer" + 0.006*"king" + 0.006*"lady" + 0.006*"torello" + '
  '0.006*"made" + 0.005*"love" + 0.005*"would" + 0.005*"might" + 0.005*"wife" '
  '+ 0.004*"great"'),
 (5,
  '0.007*"man" + 0.006*"god" + 0.006*"would" + 0.005*"ciappelletto" + '
  '0.005*"ser" + 0.005*"great" + 0.004*"good" + 0.004*"thus" +